# Pretraining GPT2

We shorten the context length because our data set doesn't have enough tokens for the original 1024 context.

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.0,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

## Torch Implementation

Import the libraries, and now we have separated the `GPTModel` implementation to a new file to make the notebook smaller, of course.

In [ ]:
import torch
import tiktoken
from scripts.gpt2_model import GPTModel

torch.manual_seed(123)

We will just instantiate the model. Since our config already has 0 drop out, we don't need to turn on `eval` mode for the model.

In [ ]:
model = GPTModel(GPT_CONFIG_124M)

Define some helpers. 

In [ ]:
from scripts.generate import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())


Let's test the model at the moment. Again we expect it to generate some garbage.

In [ ]:
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
  model=model,
  idx=text_to_token_ids(start_context, tokenizer),
  max_new_tokens=10,
  context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

## Loss Calculation

we have 2 batches here, so we add the corresponding "next" token into the targets while maintaining a consistent context length. this gives a "sliding window" effect.

In [ ]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

inputs, targets

In [ ]:
torch.manual_seed(123)

logits = model(inputs)

probas = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probas.shape) # Shape: (batch_size, num_tokens, vocab_size)


Probas contains for each token, there are vocab_size probabilities in the last dimension. We want to select the token ID (the index) of the highest number

In [ ]:

probas

For each word, we find an individual token to succeed the text

In [ ]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

But too bad! we're nowhere close to our target!

In [ ]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

print(f"Targets batch 2: {token_ids_to_text(targets[1], tokenizer)}")
print(f"Outputs batch 2: {token_ids_to_text(token_ids[1].flatten(), tokenizer)}")

So what did this probabilities look like for the _real_ indices for targets? You'll find them to be small. The goal now is to increase these probabilities.

In [ ]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)

Take the log for individual probabilities and then concatenate them.

In [ ]:
# Compute logarithm of all token probabilities
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

Take the average and negate to make it positive.

In [ ]:
# Calculate the average probability for each token
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

In [ ]:
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)


The above is basically what we call "cross entropy loss"

## Cross Entropy

Lets just use torch cross_entropy to do everything we did above.

In [ ]:
# Logits have shape (batch_size, num_tokens, vocab_size)
print("Logits shape:", logits.shape)

# Targets have shape (batch_size, num_tokens)
print("Targets shape:", targets.shape)

In [ ]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()

print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

In [ ]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

We can calculate the perplexity too.

In [ ]:
perplexity = torch.exp(loss)
print(perplexity)

## Data Stuff

Let's prepare the data for training. We had downloaded `the-verdict.txt` a while back, and now it is time to use it again.

In [9]:
with open("data/the-verdict.txt", "r") as f:
  text_data = f.read()

total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))

print("First 100 characters:", text_data[:100])
print("Characters:", total_characters)
print("Tokens:", total_tokens)

First 100 characters: I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g
Characters: 20479
Tokens: 5145


Now let's split the data into training and validation sets. We'll use 90% of the data for our training data, and the rest of the 10% as the validation data set.

To create these data loaders, let's tap into our `create_dataloader_v1` function we wrote a few notebooks back.

In [10]:
from scripts.prepare_data import create_dataloader_v1

# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

Our data set is really small, so we shouldn't have too many batches. Turns out, with `the-verdict.txt`, we only have 1 validation batch for our context length of 256.

In [12]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)

print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)
    

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


Let's verify how many tokens are in the training and validation batches individually.

In [14]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 4608
Validation tokens: 512
All tokens: 5120


## Training Loss Calculation

The main meat of the training loop involves performing calculations on the loss. We will calculate the cross-entropy loss between an input batch and target batch. 

What is returned is a tensor representing for given token, we give indication of the loss for each other token within the given vocabulary

In [15]:
def calc_loss_batch(input_batch, target_batch, model):
    input_batch, target_batch = input_batch, target_batch
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())

    return loss

`calc_loss_loader` will wrap around the `calc_loss_batch` and process for every input batch and its corresponding target batch.

In [16]:

def calc_loss_loader(data_loader, model, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

What's the current training loss and validation loss for our model? Well, since we haven't trained it yet, I wouldn't be surprised if it is really high!

In [18]:
torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

train_loss = calc_loss_loader(train_loader, model)
val_loss = calc_loss_loader(val_loader, model)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 0.28953828579849666
Validation loss: 6.4668426513671875


## Training Our Model

The time has come to train our model! Here, we will use pure CPU inferencing, and I've simplified the original code a bit to make things easier to understand.

In [ ]:
def train_model_simple(model, train_loader, val_loader, optimizer, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses = [], []
    global_step = -1

    # Main training loop
    for epoch in range(num_epochs):
        
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, start_context
        )

    return train_losses, val_losses 


def evaluate_model(model, train_loader, val_loader, eval_iter):
    train_loss = calc_loss_loader(train_loader, model, num_batches=eval_iter)
    val_loss = calc_loss_loader(val_loader, model, num_batches=eval_iter)
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, start_context):
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer)
    token_ids = generate_text_simple(
        model=model, idx=encoded,
        max_new_tokens=50, context_size=context_size
    )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))  # Compact print format

Now, let's execute the training loop. It's a good opportunity to also time the entire training. On my i7 13700, it takes a little over 2 minutes for this training data.

In [ ]:
from scripts.perf_timer import PerfTimer

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10

timer = PerfTimer()

timer.start()
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)
timer.stop()

print(f"Took this long to train: {timer.elapsed_ms()} ms")


In [ ]:
import tiktoken
from scripts.generate import generate_text_simple

tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=50,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

## Saving the Model Weights

In [ ]:
torch.save(model.state_dict(), "models/gpt2-verdict-model.pth")

## Reload the Model

In [ ]:
import torch
from scripts.gpt2_model import GPTModel

model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(
  torch.load("models/gpt2-verdict-model.pth", weights_only=True)
)
model.eval()

In [ ]:
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

## TTNN

We are not going to train the model using TTNN, instead, we will use it to perform inference in an already trained model. In this case, we diligently trained the GP2 model with our data set using CPU and had saved it to disk. We can reload those weights and reapply them to the GPTModel_ttnn class.


In [1]:
import ttnn
import tiktoken
import torch
from torch import nn
from scripts.text_helpers import text_to_token_ids, token_ids_to_text
from scripts.gpt2_model import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.0,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)

device = None

if device:
  ttnn.close_device(device)

device_id = 0
device = ttnn.open_device(device_id=device_id)

device.enable_program_cache()

model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(
  torch.load("models/gpt2-verdict-model.pth", weights_only=True)
)
model.eval()

2025-05-12 19:16:25.110 | DEBUG    | ttnn.library_tweaks:prepare_dir_as_metal_home:54 - Existing installation of 0.57.0rc60+any detected
2025-05-12 19:16:25.133 | DEBUG    | ttnn:<module>:83 - Initial ttnn.CONFIG:
Config{cache_path=/home/avgdev/.cache/ttnn,model_cache_path=/home/avgdev/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_should_raise_exception=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}


                 Device | INFO     | Opening user mode device driver
2025-05-12 19:16:25.714 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled

2025-05-12 19:16:25.722 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled
2025-05-12 19:16:25.724 | INFO     | SiliconDriver   - Harvesting mask for chip 0 is 0x200 (physical layout: 0x1, logical: 0x200, simulated harvesting mask: 0x0).
2025-05-12 19:16:25.724 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled
2025-05-12 19:16:25.725 | INFO     | SiliconDriver   - Detected PCI devices: [0]
2025-05-12 19:16:25.725 | INFO     | SiliconDriver   - Using local chip ids: {0} and remote chip ids {}
2025-05-12 19:16:25.733 | INFO     | SiliconDriver   - Software version 6.0.0, Ethernet FW version 6.14.0 (Device 0)
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | 

New chip! We now have 1 chips
Chip initialization complete (found )
Chip initializing complete...
 ARC

 [4/4] DRAM

 [16/16] ETH

 CPU

Chip detection complete (found )


GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layer): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=

## Open the Device

In [2]:
if device:
  ttnn.close_device(device)

from scripts.gpt2_model import GPTModel

device_id = 0
device = ttnn.open_device(device_id=device_id)

device.enable_program_cache()

model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(
  torch.load("models/gpt2-verdict-model.pth", weights_only=True)
)
model.eval()

                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Enabling program cache on device 0


GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layer): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=

## Initialize the GPT Model

We will reload the model to make sure that we have the latest changes from disk. 

Since we already have a model instance, we'll just need to copy over all the weights to the `ttnn` version of our model.

In [3]:
from scripts.gpt2_model_ttnn import GPTModel_ttnn, TransformerBlock_ttnn

model_ttnn = GPTModel_ttnn(GPT_CONFIG_124M, device)

model_ttnn.pos_emb.weight = torch.nn.Parameter(model.pos_emb.weight) 
model_ttnn.tok_emb.weight = torch.nn.Parameter(model.tok_emb.weight)

for i, block in enumerate(model.trf_blocks):
  t = model_ttnn.trf_blocks_ttnn[i]

  t.att.W_key.weight = torch.nn.Parameter(block.att.W_key.weight)
  t.att.W_key.bias = torch.nn.Parameter(block.att.W_key.bias)

  t.att.W_query.weight = torch.nn.Parameter(block.att.W_query.weight)
  t.att.W_query.bias = torch.nn.Parameter(block.att.W_query.bias)
  
  t.att.W_value.weight = torch.nn.Parameter(block.att.W_value.weight)
  t.att.W_value.bias = torch.nn.Parameter(block.att.W_value.bias)
  
  t.att.out_proj.weight = torch.nn.Parameter(block.att.out_proj.weight)
  t.att.out_proj.bias = torch.nn.Parameter(block.att.out_proj.bias)

  t.ff.lin_1.weight = torch.nn.Parameter(block.ff.layer[0].weight)
  t.ff.lin_1.bias = torch.nn.Parameter(block.ff.layer[0].bias)
  t.ff.lin_2.weight = torch.nn.Parameter(block.ff.layer[2].weight)
  t.ff.lin_2.bias = torch.nn.Parameter(block.ff.layer[2].bias)

  t.norm1.scale = torch.nn.Parameter(block.norm1.scale)
  t.norm1.shift = torch.nn.Parameter(block.norm1.shift)

  t.norm2.scale = torch.nn.Parameter(block.norm2.scale)
  t.norm2.shift = torch.nn.Parameter(block.norm2.shift)

model_ttnn.final_norm.shift = torch.nn.Parameter(model.final_norm.shift)
model_ttnn.final_norm.scale = torch.nn.Parameter(model.final_norm.scale)
model_ttnn.out_head.weight = torch.nn.Parameter(model.out_head.weight)

model_ttnn.update_weights()

## Out Head, Final Norm, and Transformer Blocks Comparison

In [4]:
from scripts.compare_tensors import compare_tensors

# position and token embedding comparisons
compare_tensors(
  ttnn.to_torch(
    ttnn.reshape(
      (model_ttnn.tok_emb_ttnn),
      (1, model_ttnn.tok_emb_ttnn.shape[0], model_ttnn.tok_emb_ttnn.shape[1])
    )
  ),
  model.tok_emb.weight.unsqueeze(0),
  suppress_details=True
)

compare_tensors(
  ttnn.to_torch(
    ttnn.reshape(
      (model_ttnn.pos_emb_ttnn),
      (1, model_ttnn.pos_emb_ttnn.shape[0], model_ttnn.pos_emb_ttnn.shape[1])
    )
  ),
  model.pos_emb.weight.unsqueeze(0),
  suppress_details=True
)
# outhead comparison
out_head_shape = model_ttnn.out_head_ttnn.shape
print(out_head_shape)

compare_tensors(
  ttnn.to_torch(
    ttnn.reshape(
      model_ttnn.out_head_ttnn,
      (1, out_head_shape[0], out_head_shape[1])
    )
  ),
  model.out_head.weight.unsqueeze(0),
  suppress_details=True
)

for i, block in enumerate(model.trf_blocks):
  t = model_ttnn.trf_blocks_ttnn[i]

  compare_tensors(
    ttnn.to_torch(
      ttnn.reshape(
        t.att.W_key_ttnn,
        (1, t.att.W_key_ttnn.shape[0], t.att.W_key_ttnn.shape[1]))
    ),
    model.trf_blocks[i].att.W_key.weight.unsqueeze(0),
    suppress_details=True
  )
  compare_tensors(
    ttnn.to_torch(
      ttnn.reshape(
        t.att.W_query_ttnn,
        (1, t.att.W_query_ttnn.shape[0], t.att.W_query_ttnn.shape[1]))
    ),
    model.trf_blocks[i].att.W_query.weight.unsqueeze(0),
    suppress_details=True
  )
  compare_tensors(
    ttnn.to_torch(
      ttnn.reshape(
        t.att.W_value_ttnn,
        (1, t.att.W_value_ttnn.shape[0], t.att.W_value_ttnn.shape[1]))
    ),
    model.trf_blocks[i].att.W_value.weight.unsqueeze(0),
    suppress_details=True
  )
  compare_tensors(
    ttnn.to_torch(
      ttnn.reshape(
        t.att.out_proj_ttnn,
        (1, t.att.out_proj_ttnn.shape[0], t.att.out_proj_ttnn.shape[1]))
    ),
    model.trf_blocks[i].att.out_proj.weight.unsqueeze(0),
    suppress_details=True
  )
  compare_tensors(
    ttnn.to_torch(
      ttnn.reshape(
        t.att.out_proj_bias_ttnn,
        (1, 1, t.att.out_proj_bias_ttnn.shape[0]))
    ),
    model.trf_blocks[i].att.out_proj.bias.unsqueeze(0).unsqueeze(0),
    suppress_details=True
  )

# Final Norm comparison
final_norm_shift_shape = model_ttnn.final_norm.shift_ttnn.shape
final_norm_scale_shape = model_ttnn.final_norm.scale_ttnn.shape

compare_tensors(
  ttnn.to_torch(
    ttnn.reshape(
      model_ttnn.final_norm.shift_ttnn,
      (1, 1, final_norm_shift_shape[0])
    )
  ),
  model.final_norm.shift.unsqueeze(0).unsqueeze(0),
  suppress_details=True
)

print()

compare_tensors(
  ttnn.to_torch(
    ttnn.reshape(
      model_ttnn.final_norm.scale_ttnn,
      (1, 1, final_norm_scale_shape[0])
    )
  ),
  model.final_norm.scale.unsqueeze(0).unsqueeze(0),
  suppress_details=True
)


Overall Status: ✅ PASS

Overall Status: ✅ PASS
Shape([50257, 768])

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ P

{'max_diff': 0.0,
 'mean_diff': 0.0,
 'correlation': 1.0078125,
 'max_diff_status': True,
 'mean_diff_status': True,
 'correlation_status': True,
 'overall_status': True}

## Text Generation with Loaded Weights on Tenstorrent Hardware

**! A little note about this section. !**

Unfortunately as of May 2025, `ttnn` seems to have a bit of trouble calculating the `argmax` of a tensor. It also has trouble with concatenating two tensors together too. 

Because of that, I have to move the tensors from the device memory _into_ host memory. This sucks because then we just nullify all the work we did in "accelerating" the computation for inference.

The transfer between the Wormhole and the Core i7 13700 is just too slow. Repeating this for multiple iterations of token generation, it just results in a horrible experience.

For the sake of education, I will be assuming that we do get acceleration from all this in the world where `argmax` and `concat` are fixed in `ttnn`. 

----


In [ ]:
from scripts.generate_ttnn import generate_text_simple_ttnn

torch.manual_seed(123)
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

inputs_ttnn = ttnn.from_torch(
  text_to_token_ids(start_context, tokenizer),
  dtype=ttnn.uint32,
  layout=ttnn.TILE_LAYOUT,
  device=device
)
token_ids_ttnn = generate_text_simple_ttnn(
  model=model_ttnn,
  idx_ttnn=inputs_ttnn,
  max_new_tokens=100,
  context_size=GPT_CONFIG_124M["context_length"],
  device=device
)

token_ids = ttnn.to_torch(token_ids_ttnn)


                  Metal | WARNING  | Circular buffer indices are not contiguous starting at 0. This will hurt dispatch performance. Non-contiguous indices: 16. First unused index: 1. Kernels: reader_unary_pad_dims_split_rows_multicore
                  Metal | WARNING  | Circular buffer indices are not contiguous starting at 0. This will hurt dispatch performance. Non-contiguous indices: 16. First unused index: 1. Kernels: writer_unary_interleaved_start_id, reader_unary_pad_dims_split_rows_multicore, tilize
                  Metal | WARNING  | Circular buffer indices are not contiguous starting at 0. This will hurt dispatch performance. Non-contiguous indices: 2,3. First unused index: 1. Kernels: reader_unary_reduce_interleaved_start_id
                  Metal | WARNING  | Circular buffer indices are not contiguous starting at 0. This will hurt dispatch performance. Non-contiguous indices: 2,3. First unused index: 1. Kernels: writer_unary_interleaved_start_id, reader_unary_reduce_inter

Now, the output text we get from doing all inference on the hardware should be the same as the CPU's generated text.

In [ ]:
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you?"

"Yes--quite insensible to the irony. She wanted him vindicated--and by me!"

He laughed again, and threw back his head to look up at the sketch of the donkey. "There were days when I couldn't look at that thing--couldn't face it. But I forced myself to put it here; and now it's cured me--cured me. But no--for it was not till after that event that the _rose Dubarry


Finally, close the device.

In [ ]:
ttnn.close_device(device)

## Conclusion

`ttnn` is an operator library. It's meant for raw compute of tensors. Therefore, we should leverage the library on Tenstorrent hardware for _inference_. 

There are ways we can train on the hardware, but I'm not skilled enough yet.

For the rest of the notebook series, we'll continue to use the method in training through CPU, and inferencing through Tenstorrent hardware. 